In [1]:
'''
data/train.csv:
https://github.com/ryonion/kaggle_question_pair/tree/master/data

data/pp1.csv:
https://github.com/ryonion/kaggle_question_pair/tree/master/data

data/GoogleNews-vectors-negative300.bin.gz:
https://github.com/mmihaltz/word2vec-GoogleNews-vectors

'''

import nltk, re, pprint
# from nltk.book import *
from nltk import word_tokenize
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from nltk import pos_tag
pos_idx = dict(ADJ=0, ADP=1, ADV=2, CONJ=3, DET=4, NOUN=5, NUM=6, PRT=7, PRON=8, VERB=9, X=10)
labels = ["ADJ", "ADP", "ADV", "CONJ", "DET", "NOUN", "NUM", "PRT", "PRON", "VERB", "X", "sim_avg"]


C:\Users\ryanz\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [52]:
train_data = pd.read_csv('data/train.csv')

print(list(train_data["question1"])[0:2])
print(list(train_data["question2"])[0:2])

q1 = [s.replace(",",' ').split() for s in train_data["question1"] if type(s) == str]
q2 = [s.replace(",",' ').split() for s in train_data["question2"] if type(s) == str]

['What is the step by step guide to invest in share market in india?', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?']
['What is the step by step guide to invest in share market?', 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?']


In [56]:
# w2v = Word2Vec(q1+q2)
# w2v.save("emb_tr")

# type(w2v)

gensim.models.word2vec.Word2Vec

In [269]:
# w2x = Word2Vec.load("emb_tr")
    

In [2]:
word2vec = KeyedVectors.load_word2vec_format("data/GoogleNews-vectors-negative300.bin.gz",binary=True)

In [9]:
vec_size = word2vec.vector_size

In [10]:
pproc = pd.read_csv("data/pp1.csv")

In [4]:
pproc.shape

(80000, 6)

In [7]:
pos_idx = dict(ADJ=0, ADP=1, ADV=2, CONJ=3, DET=4, NOUN=5, NUM=6, PRT=7, PRON=8, VERB=9, X=10)

def parts_of_speech(s):
        if type(s) == str:                                                 
                tokens = word_tokenize(s)
        else:
                tokens = s                                                 
        tokens_and_tags = pos_tag(tokens, tagset = "universal")
        return(tokens_and_tags)

def sent_embedding(pos_li, vec_size):
    pos_groups = [ np.zeros((vec_size)) for i in range(len(pos_idx)) ]
    for word, pos in pos_li:
        if pos != '.':
            if word in word2vec.vocab:
                pos_groups[pos_idx[pos]] += word2vec.word_vec(word)
    return pos_groups

def sent_cos_similarity(pos_group1, pos_group2):
    ret = []
    for i, _ in enumerate(pos_group1):
        ret.append((cosine_similarity(pos_group1[i].reshape(1, -1), pos_group2[i].reshape(1, -1)))[0][0])
    return tuple(ret)

def sent_similarity_group(row):
    s1 = row["question1"]
    s2 = row["question2"]
    
    pos_li_1 = parts_of_speech(s1)
    pos_li_2 = parts_of_speech(s2)
    
    # assign the words in a sentence into each of the 11 POS groups
    # substitute each word in each POS group with the word's embedding vector
    # sum the vectors in each POS group of a sentence
    # so we will have a list of 11 vectors for each sentence
    
    emb_list_1 = sent_embedding(pos_li_1, vec_size)
    emb_list_2 = sent_embedding(pos_li_2, vec_size)
    # compute the cosine_similarity of the corresponding groups in the two sentences
    # so we will have a list of 11 cosine_similarity measures.

    cos_sim = sent_cos_similarity(emb_list_1, emb_list_2)
    temp = sum(int(i>0) for i in cos_sim)
    if temp == 0: temp = 1
    perc = sum(cos_sim) / temp
    return cos_sim + (perc,)

#### test sent_similarity_group

In [74]:
example = ["hello good morning","hello good night"]

In [75]:
s = pd.Series(example, index=['question1','question2'])

In [76]:
print([ (a,b) for (a,b) in zip(labels, sent_similarity_group(s))])

[('ADJ', 1.0000000000000002), ('ADP', 0.0), ('ADV', 0.9999999999999999), ('CONJ', 0.0), ('DET', 0.0), ('NOUN', 0.6452910623105599), ('NUM', 0.0), ('PRT', 0.0), ('PRON', 0.0), ('VERB', 0.0), ('X', 0.0), ('sim_avg', 0.8817636874368532)]


In [11]:
emb = pproc[40000:].apply(sent_similarity_group,axis=1, raw=True )

In [12]:
emb.head(3)

40000    (0.0, 0.0, 1.0000000000000002, 0.0, 0.0, 0.661...
40001    (0.9999999999999999, 0.9220422233390738, 0.0, ...
40002    (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.311...
dtype: object

In [13]:
emb_li = emb.tolist()

In [14]:
df = pd.DataFrame.from_records(emb_li, columns=labels)

In [16]:
#df.to_csv("data/emb_mat_test.csv", index=False)